# Homework 2-Question-1
Stigler’s diet. True story! In 1945, American economist (and future Nobel laureate) George
Stigler published a paper investigating the composition of an optimal diet; minimizing total cost while
meeting the recommended daily allowance (RDA) of several nutrients. To answer this question, Stigler
tabulated a list of 77 foods and their nutrient content for 9 nutrients: calories, protein, calcium, iron,
vitamin A, thiamine, riboflavin, niacin, and ascorbic acid.

Formulate Stigler’s diet problem as an LP and solve it. To get you started, Stigler’s original data
is provided in stigler.csv, and the IJulia notebook stigler.ipynb imports the data and puts it
into a convenient array format. How does your cheapest diet compare in annual cost to Stigler’s?
What foods make up your optimal diet?

## Load Data

In [1]:
using NamedArrays      # make sure you run Pkg.add("NamedArrays") first!

# import Stigler's data set
raw = readcsv("stigler.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );

## Problem Model

In [2]:
using JuMP

m = Model()

# Variable vector for quantities of food
@variable(m, quantities[foods] >= 0)

# expression for cost of total food. Since the quantites are per unit dollar, summation of quantities
# is sufficient to calculate the total cost.
@expression(m, cost, sum(quantities[f] for f in foods))

# constraint to lower bound the quantity of nutrients required
@constraint(m, min_req[n in nutrients],sum(data[f,n]*quantities[f] for f in foods) >= lower[n])

# Objective to minimize the total cost
@objective(m, Min, cost)

solve(m)

[println(getvalue(quantities[f]), "\t\t", f) for f in foods if getvalue(quantities)[f] > 0]
println("Minimum Cost yearly: \$", getobjectivevalue(m)*365.25)

0.02951906167648827		Wheat Flour (Enriched)
0.0018925572907052643		Liver (Beef)
0.011214435246144865		Cabbage
0.005007660466725203		Spinach
0.061028563526693246		Navy Beans, Dried
Minimum Cost yearly: $39.68889711501794


## Vegan and Gluten Free Diet
I will add the following constraints to the model created above. This constraints are only applied seeing the minimum cost diet solution given by the solver above.
- Liver(Beef) : Comes from animals so it's not vegan.
- Wheat Flour (Enriched) : Contains gluten.
- Lard : Animal fat

So the quantities of these are constrained to be zero.

In [9]:
non_vegan = ["Wheat Flour (Enriched)", "Liver (Beef)", "Lard"]
for nv in non_vegan
    @constraint(m, [nv], quantities[nv] == 0)
end
solve(m)

[println(getvalue(quantities[f]), "\t\t", f) for f in foods if getvalue(quantities)[f] > 0]
println("Minimum Cost yearly: \$", getobjectivevalue(m)*365.25)

0.005344246335991787		Corn Meal
0.01131324508827593		Cabbage
0.00517574850128731		Spinach
0.10306689112726254		Navy Beans, Dried
Minimum Cost yearly: $45.61977286704162


So the minimum cost increases from \$39.69 to \$45.62 annualy for vegan and gluten-free diet